- **scene/gaussian_model.py**
    - 생성자에서 정적 파라미터(예: static_xyz, static_features_dc 등)를 초기화
    - `capture()`와 `restore()` 함수에 정적 파라미터 포함
    - `separate_static_gaussians`(tau) (또는 convert_to_3d(indices)) 함수를 구현하여, exp⁡(st)>τ\exp(s_t) > \tauexp(st)>τ인 4D Gaussian을 3D로 전환
    - `get_combined_means`(), `get_combined_covariance`() (및 필요 시 get_all_opacities(), get_all_sh_features()) 함수를 추가하여 동적과 정적 데이터를 하나로 결합
- **train.py**
    - 학습 루프 내에서 densification 또는 opacity reset 후 주기적으로 `separate_static_gaussians`(tau)를 호출하여 정적 Gaussian 전환을 수행
- **gaussian_renderer (예: init.py)**
    - 렌더링 시 모델의 통합 getter 함수들을 사용하여, 동적 및 정적 Gaussian의 데이터를 하나로 결합한 후 투영, 깊이 정렬, 그리고 alpha blending을 수행

# 1. scene/gaussian_model.py

In [ ]:
class GaussianModel:
    def __init__(self, sh_degree: int, gaussian_dim: int = 3, time_duration: list = [-0.5, 0.5],
                 rot_4d: bool = False, force_sh_3d: bool = False, sh_degree_t: int = 0):
        # 기존 4D 파라미터 초기화 (예시)
        self.gaussian_dim = gaussian_dim
        self._xyz = torch.empty(0)
        self._features_dc = torch.empty(0)
        self._features_rest = torch.empty(0)
        self._scaling = torch.empty(0)
        self._rotation = torch.empty(0)
        self._opacity = torch.empty(0)
        self.max_radii2D = torch.empty(0)
        self.xyz_gradient_accum = torch.empty(0)
        self.denom = torch.empty(0)
        self.optimizer = None
        self.percent_dense = 0
        self.spatial_lr_scale = 0

        self._t = torch.empty(0)
        self._scaling_t = torch.empty(0)
        self.time_duration = time_duration
        self.rot_4d = rot_4d
        self._rotation_r = torch.empty(0)
        self.force_sh_3d = force_sh_3d
        self.t_gradient_accum = torch.empty(0)
        self.env_map = torch.empty(0)

        self.active_sh_degree = 0
        self.max_sh_degree = sh_degree
        self.active_sh_degree_t = 0
        self.max_sh_degree_t = sh_degree_t

        self.setup_functions()

        # ===== 추가: 정적 Gaussian 파라미터 초기화 =====
        self.static_xyz = torch.empty(0)
        self.static_features_dc = torch.empty(0)
        self.static_features_rest = torch.empty(0)
        self.static_scaling = torch.empty(0)
        self.static_rotation = torch.empty(0)
        self.static_opacity = torch.empty(0)
        self.static_max_radii2D = torch.empty(0)
        self.static_xyz_gradient_accum = torch.empty(0)
        # =================================================

def capture(self):
    if self.gaussian_dim == 4:
        return (
            self.active_sh_degree,
            self._xyz,
            self._features_dc,
            self._features_rest,
            self._scaling,
            self._rotation,
            self._opacity,
            self.max_radii2D,
            self.xyz_gradient_accum,
            self.t_gradient_accum,
            self.denom,
            self.optimizer.state_dict(),
            self.spatial_lr_scale,
            self._t,
            self._scaling_t,
            self._rotation_r,
            self.rot_4d,
            self.env_map,
            self.active_sh_degree_t,
            # ===== 추가: 정적 파라미터 =====
            self.static_xyz,
            self.static_features_dc,
            self.static_features_rest,
            self.static_scaling,
            self.static_rotation,
            self.static_opacity,
            self.static_max_radii2D,
            self.static_xyz_gradient_accum
            # =================================
        )
    else:
        # 3D인 경우는 기존 방식 사용
        pass

def restore(self, model_args, training_args):
    if self.gaussian_dim == 4:
        (self.active_sh_degree,
         self._xyz,
         self._features_dc,
         self._features_rest,
         self._scaling,
         self._rotation,
         self._opacity,
         self.max_radii2D,
         self.xyz_gradient_accum,
         self.t_gradient_accum,
         self.denom,
         opt_dict,
         self.spatial_lr_scale,
         self._t,
         self._scaling_t,
         self._rotation_r,
         self.rot_4d,
         self.env_map,
         self.active_sh_degree_t,
         # ===== 추가: 정적 파라미터 복원 =====
         self.static_xyz,
         self.static_features_dc,
         self.static_features_rest,
         self.static_scaling,
         self.static_rotation,
         self.static_opacity,
         self.static_max_radii2D,
         self.static_xyz_gradient_accum
         # ==================================
        ) = model_args
    else:
        # 3D인 경우는 기존 방식 사용
        pass

    if training_args is not None:
        self.training_setup(training_args)
        self.xyz_gradient_accum = self.xyz_gradient_accum
        self.t_gradient_accum = self.t_gradient_accum
        self.denom = self.denom
        self.optimizer.load_state_dict(opt_dict)


def separate_static_gaussians(self, tau):
    """
    4D Gaussian 중 _scaling_t의 exp() 값이 tau를 초과하면 해당 Gaussian을 정적으로 전환합니다.
    전환 시, 시간 성분(_t)을 0으로 설정하고, 4D 회전 행렬에서 공간 부분만 추출하며, 
    시간 관련 파라미터(_scaling_t, 시간 SH 등)는 고정합니다.
    """
    if self.gaussian_dim != 4:
        print("모델이 3D이므로 정적/동적 분리는 적용되지 않습니다.")
        return

    temporal_scales = torch.exp(self._scaling_t).squeeze(-1)  # (N,)
    static_mask = temporal_scales > tau
    dynamic_mask = ~static_mask

    num_static = static_mask.sum().item()
    num_dynamic = dynamic_mask.sum().item()
    print(f"정적 Gaussian 개수: {num_static}, 동적 Gaussian 개수: {num_dynamic}")
    if num_static == 0:
        return

    # 정적 Gaussian 파라미터를 추가
    self.static_xyz = torch.cat([self.static_xyz, self._xyz[static_mask]], dim=0)
    self.static_features_dc = torch.cat([self.static_features_dc, self._features_dc[static_mask]], dim=0)
    self.static_features_rest = torch.cat([self.static_features_rest, self._features_rest[static_mask]], dim=0)
    self.static_scaling = torch.cat([self.static_scaling, self._scaling[static_mask]], dim=0)
    self.static_rotation = torch.cat([self.static_rotation, self._rotation[static_mask]], dim=0)
    self.static_opacity = torch.cat([self.static_opacity, self._opacity[static_mask]], dim=0)
    self.static_max_radii2D = torch.cat([self.static_max_radii2D, self.max_radii2D[static_mask]], dim=0)
    self.static_xyz_gradient_accum = torch.cat([self.static_xyz_gradient_accum, self.xyz_gradient_accum[static_mask]], dim=0)

    # 동적 Gaussian 집합에서 정적 항목 제거 (시간 관련 파라미터도 함께 제거)
    self._xyz = self._xyz[dynamic_mask]
    self._features_dc = self._features_dc[dynamic_mask]
    self._features_rest = self._features_rest[dynamic_mask]
    self._scaling = self._scaling[dynamic_mask]
    self._rotation = self._rotation[dynamic_mask]
    self._opacity = self._opacity[dynamic_mask]
    self.max_radii2D = self.max_radii2D[dynamic_mask]
    self.xyz_gradient_accum = self.xyz_gradient_accum[dynamic_mask]
    self._t = self._t[dynamic_mask]
    self._scaling_t = self._scaling_t[dynamic_mask]
    if self.rot_4d:
        self._rotation_r = self._rotation_r[dynamic_mask]

def get_combined_means(self, timestamp):
    """
    주어진 timestamp에서 동적 Gaussian은 시간 offset을 적용하여 3D 좌표를 계산하고,
    정적 Gaussian은 이미 3D 상태이므로 그대로 반환한 후, 두 그룹을 합쳐 반환한다.
    """
    dynamic_means = self._xyz.clone()
    if self.gaussian_dim == 4:
        _, delta_mean = self.get_current_covariance_and_mean_offset(1.0, timestamp)
        if hasattr(self, 'static_mask') and self.static_mask.any():
            delta_mean[self.static_mask] = 0.0
        dynamic_means += delta_mean
    # 정적 Gaussian의 좌표는 self.static_xyz에 저장되어 있음
    static_means = self.static_xyz
    return torch.cat([dynamic_means, static_means], dim=0)

def get_combined_covariance(self, scaling_modifier=1.0, timestamp=None):
    """
    주어진 timestamp에서 동적 Gaussian과 정적 Gaussian의 3×3 공분산을 결합하여 반환한다.
    """
    if self.gaussian_dim == 4:
        cov, _ = self.get_current_covariance_and_mean_offset(scaling_modifier, timestamp)
        if hasattr(self, 'static_mask') and self.static_mask.any():
            static_idx = self.static_mask
            cov_static = GaussianModel.build_covariance_from_scaling_rotation(
                scaling_modifier * self._scaling[static_idx],
                torch.eye(3, device=cov.device).reshape(1, 3, 3).repeat(static_idx.sum(), 1, 1)
            )
            cov[static_idx] = cov_static
        dynamic_cov = cov
    else:
        dynamic_cov = self.get_covariance(scaling_modifier)
    
    # 정적 Gaussian의 공분산 계산 (이미 3D이므로)
    static_cov = GaussianModel.build_covariance_from_scaling_rotation(
        self.static_scaling.exp(),  # scaling_activation 적용된 값
        self.static_rotation
    )
    return torch.cat([dynamic_cov, static_cov], dim=0)


# # ???
# def get_all_opacities(self):
#     dynamic_opacity = self.opacity_activation(self._opacity)
#     static_opacity = self.opacity_activation(self.static_opacity)
#     return torch.cat([dynamic_opacity, static_opacity], dim=0)

# def get_all_sh_features(self):
#     dynamic_sh = torch.cat((self._features_dc, self._features_rest), dim=2)
#     static_sh = torch.cat((self.static_features_dc, self.static_features_rest), dim=2)
#     return torch.cat([dynamic_sh, static_sh], dim=0)



# 2. train.py

In [ ]:
# train.py (학습 루프 내, densification 단계 후)
if iteration > opt.initial_warmup and iteration % opt.static_conversion_interval == 0:
    # 예: opt.static_conversion_interval = 100, opt.initial_warmup = 500
    # tau는 예를 들어 3.0 (또는 데이터셋에 맞게 조정)
    model.separate_static_gaussians(tau)

# 학습 루프 내에서 densification이나 opacity reset 후, 주기적으로 정적/동적 분리 함수를 호출하여 4D Gaussian 중 exp(st) > tau 인 항목을 3D로 전환합니다.

# 3. gaussian_renderer/init.py

설명:

렌더링 함수에서 동적와 정적 Gaussian 데이터를 모두 결합한 후, 단일 GPU rasterizer에서 투영, 정렬, 그리고 alpha blending을 수행합니다.

동적 Gaussian와 정적 Gaussian의 3D 좌표, 공분산, 불투명도, SH 특징을 각각 통합(get_combined_means, get_combined_covariance, get_all_opacities, get_all_sh_features)하여 하나의 리스트로 합칩니다.

이를 GPU rasterizer에 전달하면, 동일한 투영 및 alpha blending 과정을 통해 하나의 최종 이미지가 생성됩니다.

In [ ]:
# __init__.py 또는 렌더링 함수 내
# 'pc'는 GaussianModel 인스턴스, 'viewpoint_camera.timestamp'는 현재 렌더링 시간

# 1. 모든 Gaussian의 3D 좌표 결합
means_all = pc.get_combined_means(viewpoint_camera.timestamp)  # [N_dynamic + N_static, 3]

# 2. 모든 Gaussian의 공분산 결합
cov_all = None
if pipe.compute_cov3D_python:
    cov_all = pc.get_combined_covariance(scaling_modifier, viewpoint_camera.timestamp)

# 3. 불투명도 및 SH 특징도 통합 (필요 시)
opacity_all = pc.get_all_opacities()  # 동적 + 정적
sh_all = eval_shfs_4d(pc.active_sh_degree, pc.active_sh_degree_t, pc.get_features(), 
                      viewpoint_camera.get_ray_directions(), t=viewpoint_camera.timestamp)
# 4. rasterization: 기존 rasterizer에 통합 데이터를 넘겨서 렌더링
raster_settings = GaussianRasterizationSettings(
    image_height = viewpoint_camera.image_height,
    image_width = viewpoint_camera.image_width,
    tanfovx = math.tan(viewpoint_camera.FoVx * 0.5),
    tanfovy = math.tan(viewpoint_camera.FoVy * 0.5),
    bg = bg_color_tensor,
    scale_modifier = scaling_modifier,
    viewmatrix = viewpoint_camera.world_view_transform,
    projmatrix = viewpoint_camera.full_proj_transform,
    sh_degree = pc.active_sh_degree,
    sh_degree_t = pc.active_sh_degree_t,
    campos = viewpoint_camera.camera_center,
    timestamp = viewpoint_camera.timestamp,
    time_duration = pc.time_duration[1] - pc.time_duration[0],
    rot_4d = pc.rot_4d,
    gaussian_dim = pc.gaussian_dim,
    force_sh_3d = pc.force_sh_3d,
    prefiltered = False,
    opa_threshold = pipe.opa_threshold,
    debug = pipe.debug
)
rasterizer = GaussianRasterizer(raster_settings)
image = rasterizer(means3D = means_all, cov3D_precomp = cov_all, colors = sh_all, opacity = opacity_all)

